In [1]:
import os
import sys

import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt


import nltk
import random
import re

# Download NLTK resources (if not downloaded)
nltk.download('punkt')

from utils import *

[nltk_data] Downloading package punkt to C:\Users\Simon
[nltk_data]     Andersen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Simon
[nltk_data]     Andersen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
# generate csv data file for each label
dir_data = 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data'
exclude = ['cleaned', 'scrape']
# find all subdirectories and subsubdirectories
dir_paths = [os.path.join(dir_data, f) for f in os.listdir(dir_data) if os.path.isdir(os.path.join(dir_data, f))]

# add subdirectories not already in dir_paths
dirs_to_pop = []
for dir_path in dir_paths:
    sub_dir_paths = [os.path.join(dir_path, f) for f in os.listdir(dir_path)\
                      if os.path.isdir(os.path.join(dir_path, f))]

    if len(sub_dir_paths) > 0:
        dir_paths.extend(sub_dir_paths) 
        dirs_to_pop.append(dir_path)

for dir_to_pop in dirs_to_pop:
    dir_paths.remove(dir_to_pop)

# remove directories that are not to be included
pop_list = []
for dir_path in dir_paths:
    for exc in exclude:
        if dir_path.endswith(exc):
            pop_list.append(dir_path)
            break
for pop in pop_list:
    dir_paths.remove(pop)   



In [31]:
dir_paths

['C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\brevkasse',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\from_big_data_file',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\heste-nettet-nyheder',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\journalistik_interview',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\public_information',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\brevkasse_ai\\gpt-3.5-turbo-0613',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\from_big_data_file_ai\\gpt-3.5-turbo',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\from_big_data_file_ai\\gpt-3.5-turbo-0613',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\heste-nettet-nyheder-ai\\gpt-3.5-turbo',
 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiT

In [32]:
ai_endswith = ['ai', 'turbo', '0613']

dirs_ai = []
dirs_human = []
for dir_path in dir_paths:
    for end in ai_endswith:
        if dir_path.endswith(end):
            dirs_ai.append(dir_path)
            break
    if dir_path not in dirs_ai:
        dirs_human.append(dir_path)
            

In [35]:
# find all txt files in the directory
paths_human = []
paths_ai = []
for dir_hum, dir_ai in zip(dirs_human,dirs_ai):
    paths_human.extend(find_txt_files(dir_hum))
    paths_ai.extend(find_txt_files(dir_ai))

Nai = len(paths_ai)
Nhuman = len(paths_human)

paths_human = paths_human[:Nai]
Nhuman = len(paths_human)   
print(f"Number of human files: {Nhuman}")
print(f"Number of ai files: {Nai}")

Number of human files: 6104
Number of ai files: 6104


In [36]:
text_human = read_human_files(paths_human)
text_ai = read_files(paths_ai, line_start = 3)


In [39]:
# create a dataframe with the text and label (human = 0 or ai = 1) for each text list
df_human = pd.DataFrame({'text': text_human, 'label': [0]*len(text_human)})
df_ai = pd.DataFrame({'text': text_ai, 'label': [1]*len(text_ai)})

df= pd.concat([df_human, df_ai], ignore_index=True)

# shuffle the dataframe
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

text = df['text'].astype(str)
labels = df['label'].astype(int)

df = pd.DataFrame({'text': text, 'label': labels})

# remove \n from text
df['text'] = df['text'].apply(lambda x: re.sub(r'\n', ' ', x))
# remove \u200b\u200b from text
df['text'] = df['text'].apply(lambda x: re.sub(r'\u200b\u200b', ' ', x))
# remove \\\x00 from text (also if it occurs more than once)
df['text'] = df['text'].apply(lambda x: re.sub(r'\\\x00', ' ', x))

# save the dataframe as a csv file
df.to_csv('data_processed\\data_all.csv', index=False)